<a href="https://colab.research.google.com/github/Gabrieldiasdeoliveira/cross_up_selling/blob/main/cross_up_selling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Nesse notebock vou desenvolver uma analise em cima de um dataset que contém informações sobre transações de compras em um supermercado online, incluindo quais produtos foram comprados juntos.
A idéia é analisar cross-selling e upselling com base nos padrões de compras dos clientes.*


#Dowload das bases do kaggle

Nessa etava vamos acessar o dataset e fazer o dowload das bases

In [6]:
from google.colab import files

In [7]:
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"gabrielestatistico","key":"0c5e3e674729d6c445c0e7b78b9c74ea"}'}

In [8]:
pip install kaggle -q

In [9]:
!mkdir -p ~/.kaggle

In [10]:
!cp kaggle.json ~/.kaggle/

In [11]:
#!mv kaggle.json ~/.kaggle/

In [12]:
!chmod 600 ~/.kaggle/kaggle.json

In [13]:
!kaggle competitions download -c instacart-market-basket-analysis


instacart-market-basket-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [14]:
# extraindo todos os arquivos do dataset
!unzip -o instacart-market-basket-analysis.zip


Archive:  instacart-market-basket-analysis.zip
  inflating: aisles.csv.zip          
  inflating: departments.csv.zip     
  inflating: order_products__prior.csv.zip  
  inflating: order_products__train.csv.zip  
  inflating: orders.csv.zip          
  inflating: products.csv.zip        
  inflating: sample_submission.csv.zip  


In [15]:
# Descompactando os arquivos
import zipfile
import os

# Lista de arquivos .csv.zip para descompactar
zip_files = [
    "aisles.csv.zip",
    "departments.csv.zip",
    "order_products__prior.csv.zip",
    "order_products__train.csv.zip",
    "orders.csv.zip",
    "products.csv.zip",
    "sample_submission.csv.zip"
]

# Descompacta cada arquivo .csv.zip
for file in zip_files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall()


In [17]:
import pandas as pd

# Leitura dos arquivos CSV
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
sample_submission = pd.read_csv('sample_submission.csv')




# Visualização

In [26]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("aisles:")
print(aisles.head(), "\n")


aisles:
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation 



In [19]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("departments:")
print(departments.head(), "\n")

departments:
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol 



In [20]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("order_products_prior:")
print(order_products_prior.head(), "\n")

order_products_prior:
   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0 



In [21]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("order_products_train:")
print(order_products_train.head(), "\n")

order_products_train:
   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1 



In [22]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("orders:")
print(orders.head(), "\n")

orders:
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0   



In [23]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("products:")
print(products.head(), "\n")

products:
   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13   



In [24]:
# Exibindo as primeiras linhas de cada DataFrame para verificar
print("sample_submission:")
print(sample_submission.head(), "\n")

sample_submission:
   order_id     products
0        17  39276 29259
1        34  39276 29259
2       137  39276 29259
3       182  39276 29259
4       257  39276 29259 



# Analise Exploratória de dados

*Nessa etapa faremos algumas analises descritivas, categorizações e clusterizações para entender alguns padrões nos dados.*

*Inicialmente vamos entender a recorrencia nas vendas dos produtos*


In [27]:
# Passo 1: Realizar a junção
merged_data = pd.merge(products, aisles, on='aisle_id', how='left')

# Passo 2: Contar produtos por aisle
aisle_counts = merged_data['aisle'].value_counts().reset_index()
aisle_counts.columns = ['aisle', 'product_count']  # Renomeia as colunas para melhor legibilidade

# Exibir contagem por aisle
print("Contagem de produtos por aisle:")
print(aisle_counts)

Contagem de produtos por aisle:
                            aisle  product_count
0                         missing           1258
1                 candy chocolate           1246
2                   ice cream ice           1091
3            vitamins supplements           1038
4                          yogurt           1026
..                            ...            ...
129                  frozen juice             47
130              baby accessories             44
131              packaged produce             32
132  bulk grains rice dried goods             26
133  bulk dried fruits vegetables             12

[134 rows x 2 columns]


In [28]:
merged_with_departments = pd.merge(merged_data, departments, on='department_id', how='left')

# Agora, contar produtos por department
department_counts = merged_with_departments['department'].value_counts().reset_index()
department_counts.columns = ['Departamento', 'Quantidade de Produtos']

# Exibir contagem por department
print("\nContagem de produtos por department:")
print(department_counts)


Contagem de produtos por department:
       Departamento  Quantidade de Produtos
0     personal care                    6563
1            snacks                    6264
2            pantry                    5371
3         beverages                    4365
4            frozen                    4007
5        dairy eggs                    3449
6         household                    3085
7      canned goods                    2092
8   dry goods pasta                    1858
9           produce                    1684
10           bakery                    1516
11             deli                    1322
12          missing                    1258
13    international                    1139
14        breakfast                    1115
15           babies                    1081
16          alcohol                    1054
17             pets                     972
18     meat seafood                     907
19            other                     548
20             bulk                   

In [29]:
# Passo 2: Realizar a junção
merged_data = pd.merge(products, aisles, on='aisle_id', how='left')
merged_with_departments = pd.merge(merged_data, departments, on='department_id', how='left')
merged_with_orders = pd.merge(merged_with_departments, order_products_prior, on='product_id', how='left')

# Passo 3: Agrupar e contar as vendas por departamento e produto
product_sales = merged_with_orders.groupby(['department', 'product_name'])['order_id'].count().reset_index()

# Passo 4: Renomear a coluna para melhor legibilidade
product_sales.columns = ['department', 'product_name', 'total_sales']

# Passo 5: Ordenar os produtos por departamento e total de vendas
product_sales = product_sales.sort_values(by=['department', 'total_sales'], ascending=[True, False])

# Passo 6: Selecionar os produtos mais vendidos em cada departamento
top_products_by_department = product_sales.groupby('department').head(10)


Produtos mais vendidos por departamento:
      department                                  product_name  total_sales
826      alcohol                               Sauvignon Blanc         8246
242      alcohol                                    Chardonnay         6153
179      alcohol                            Cabernet Sauvignon         6115
81       alcohol                                          Beer         5844
1005     alcohol                                         Vodka         5475
...          ...                                           ...          ...
44419     snacks                        Chocolate Chip Cookies        14148
47490     snacks                        Organic Tortilla Chips        13776
43731     snacks  Baked Aged White Cheddar Rice and Corn Puffs        13691
47101     snacks        Organic Bunny Fruit Snacks Berry Patch        13372
48557     snacks                         Sea Salt Potato Chips        13086

[210 rows x 3 columns]


In [30]:
top_products_by_department

,department,product_name,total_sales
826,alcohol,Sauvignon Blanc,8246
242,alcohol,Chardonnay,6153
179,alcohol,Cabernet Sauvignon,6115
81,alcohol,Beer,5844
1005,alcohol,Vodka,5475
...,...,...,...
44419,snacks,Chocolate Chip Cookies,14148
47490,snacks,Organic Tortilla Chips,13776
43731,snacks,Baked Aged White Cheddar Rice and Corn Puffs,13691
47101,snacks,Organic Bunny Fruit Snacks Berry Patch,13372


In [31]:
# Inspecionar as colunas do DataFrame merged_with_orders
print("Colunas do DataFrame merged_with_orders:")
print(merged_with_orders.columns)


Colunas do DataFrame merged_with_orders:
Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'aisle',
       'department', 'order_id', 'add_to_cart_order', 'reordered'],
      dtype='object')


In [32]:
import pandas as pd

In [3]:
# Passo 1: Importar a biblioteca pandas
import pandas as pd

# Passo 2: Carregar os dados
orders_prior = pd.read_csv('order_products__prior.csv')
orders_train = pd.read_csv('order_products__train.csv')


In [4]:

# Passo 3: Concatenar os dados de vendas
orders_combined = pd.concat([orders_prior, orders_train], ignore_index=True)


In [6]:
import pandas as pd

# Passo 1: Carregar os dados
products = pd.read_csv('products.csv')          # Carregue o arquivo com dados de produtos
aisles = pd.read_csv('aisles.csv')              # Carregue o arquivo com dados de corredores (aisles)
departments = pd.read_csv('departments.csv')    # Carregue o arquivo com dados de departamentos
orders_prior = pd.read_csv('order_products__prior.csv')  # Carregue os pedidos anteriores
orders_train = pd.read_csv('order_products__train.csv')  # Carregue os pedidos de treino

# Passo 2: Concatenar os dados de vendas
orders_combined = pd.concat([orders_prior, orders_train], ignore_index=True)


In [8]:
# Passo 4: Realizar as junções necessárias
merged_data = pd.merge(products, aisles, on='aisle_id', how='left')
merged_with_departments = pd.merge(merged_data, departments, on='department_id', how='left')
merged_with_orders = pd.merge(merged_with_departments, orders_combined, on='product_id', how='left')

In [26]:
# Incluir user_id no merged_with_orders
if 'user_id' not in merged_with_orders.columns:
    orders = pd.read_csv('orders.csv')
    merged_with_orders = pd.merge(merged_with_orders, orders[['order_id', 'user_id']], on='order_id', how='left')

In [27]:
# Inspecionar as colunas do DataFrame merged_with_orders
print("Colunas do DataFrame merged_with_orders:")
print(merged_with_orders.columns)

Colunas do DataFrame merged_with_orders:
Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'aisle',
       'department', 'order_id', 'add_to_cart_order', 'reordered', 'user_id'],
      dtype='object')


In [9]:
# Passo 5: Agrupar e contar as vendas por departamento e produto
product_sales = merged_with_orders.groupby(['department', 'product_id', 'product_name'])['order_id'].count().reset_index()
product_sales.columns = ['department', 'product_id', 'product_name', 'total_sales']

In [10]:
# Passo 6: Ordenar os produtos por departamento e total de vendas
product_sales = product_sales.sort_values(by=['department', 'total_sales'], ascending=[True, False])

In [11]:
# Passo 7: Selecionar os produtos mais vendidos em cada departamento
top_products_by_department = product_sales.groupby('department').head(10)

In [23]:
# Passo 8: Contar compradores únicos para cada produto
unique_buyers = merged_with_orders.groupby('product_id')['order_id'].nunique().reset_index()
unique_buyers.columns = ['product_id', 'unique_buyers']

In [13]:
# Passo 9: Contar produtos reordenados e comprados pela primeira vez
reorder_count = merged_with_orders.groupby('product_id')['reordered'].sum().reset_index()
reorder_count.columns = ['product_id', 'reorder_count']


In [20]:
# Passo 10: Calcular a porcentagem de reordenação
top_products_by_department['reorder_percentage'] = (top_products_by_department['reorder_count'] / top_products_by_department['total_sales']) * 100

In [14]:
# Passo 10: Adicionar os dados de compradores únicos e recompra ao DataFrame dos produtos mais vendidos
top_products_by_department = top_products_by_department.merge(unique_buyers, on='product_id', how='left')
top_products_by_department = top_products_by_department.merge(reorder_count, on='product_id', how='left')

In [19]:
# Exibir os resultados finais
print("\nProdutos mais vendidos por departamento com contagem de compradores únicos e recompra:")


Produtos mais vendidos por departamento com contagem de compradores únicos e recompra:


In [21]:
top_products_by_department

,department,product_id,product_name,total_sales,unique_buyers,reorder_count,reorder_percentage
0,alcohol,2120,Sauvignon Blanc,8541,8541,5847.0,68.458026
1,alcohol,33065,Cabernet Sauvignon,6352,6352,3836.0,60.390428
2,alcohol,38444,Chardonnay,6346,6346,4367.0,68.815002
3,alcohol,46088,Beer,6068,6068,4069.0,67.056691
4,alcohol,45190,Vodka,5666,5666,4103.0,72.414402
...,...,...,...,...,...,...,...
205,snacks,40199,Chocolate Chip Cookies,14762,14762,8866.0,60.059613
206,snacks,35561,Organic Tortilla Chips,14413,14413,9048.0,62.776660
207,snacks,3599,Baked Aged White Cheddar Rice and Corn Puffs,14237,14237,9168.0,64.395589
208,snacks,5194,Organic Bunny Fruit Snacks Berry Patch,13847,13847,8246.0,59.550805
